In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn. metrics import roc_curve, auc
import random
from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibrationDisplay
from sklearn. metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
def calculate_metrics(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    tn = np.sum(np.logical_and(y_true == 0, y_pred == 0))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    predictive_accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return sensitivity, specificity, ppv, npv, predictive_accuracy, mcc

In [11]:
data = pd.read_excel("C:\\Users\\12292\\Desktop\\train size - xx (2).xlsx")

In [12]:
train_X = data.iloc[:, 1:4]
train_y = data.iloc[:, 0] 

In [13]:
train_X

,BNF,PHASES,Radscore
0,1.260731,4,0.446201
1,1.385450,9,2.553235
2,1.410795,0,-1.116113
3,0.930106,0,-1.127136
4,1.066182,0,-1.669537
...,...,...,...
328,1.266495,1,-0.748227
329,1.018247,1,-1.270338
330,0.658511,5,0.754189
331,1.007611,6,-0.795665


In [14]:
train_y 

0      1
1      1
2      0
3      0
4      0
      ..
328    0
329    0
330    1
331    0
332    1
Name: status, Length: 333, dtype: int64

In [ ]:
# # In R, the predicted probabilities from LR(R, M, and P) has been saved.
# Ensure that the order of variables is consistent with the order used in the Excel file previously utilized for model training.
# This needs to be sorted and processed in Excel.

In [15]:
data_test = pd.read_excel("C:\\Users\\12292\\Desktop\\merged_data_with_predictions.xlsx")

In [16]:
test_y = data_test.iloc[:, 0] 

In [17]:
test_X = data_test.iloc[:, 1:4] 
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [18]:
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [19]:
test_y

0      0
1      0
2      0
3      1
4      0
      ..
137    0
138    0
139    1
140    0
141    1
Name: status, Length: 142, dtype: int64

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

rf_params = {
    'n_estimators': 210,
    'random_state':9999,
    'max_depth':3,
    'min_samples_leaf':34,
    'min_samples_split':70,
}

best_RF = RandomForestClassifier(**rf_params)

best_RF.fit(train_X, train_y)
RF_y_pred = best_RF.predict(test_X)
accuracy = accuracy_score(test_y, RF_y_pred)
print("Test Accuracy with Best Parameters:", accuracy)

Test Accuracy with Best Parameters: 0.795774647887324


In [21]:
accuracy = accuracy_score(test_y,  RF_y_pred)
RF_y_score = best_RF.predict_proba(test_X)[:, 1]
fprrf,tprrf,threshold = roc_curve(test_y,RF_y_score)
auc(fprrf,tprrf)

0.8803980099502487

In [22]:
from sklearn.metrics import roc_auc_score, roc_curve

# Predict on the training set
RF_train_y_pred = best_RF.predict(train_X)
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

# Calculate AUC
train_auc = roc_auc_score(train_y, train_y_prob)

# Calculate ROC curve and AUC for the original data
fprRF, tprRF, thresholds = roc_curve(train_y, train_y_prob)
original_auc = auc(fprRF, tprRF)
print("Original AUC: {:.3f}".format(original_auc))

Original AUC: 0.916


In [23]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
best_model_XGB = xgb.XGBClassifier(n_estimators=254, learning_rate=0.0395, max_depth=3, min_child_weight=7,colsample_bytree=0.4, 
                                   subsample=0.4,random_state=527)

best_model_XGB.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0395, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              n_estimators=254, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=527, ...)

In [25]:
XGB_train_y_pred = best_model_XGB.predict(train_X)
XGBboost_train_y_score = best_model_XGB.predict_proba(train_X)[:, 1]
fprXGB,tprXGB,threshold = roc_curve(train_y,XGBboost_train_y_score)
auc(fprXGB,tprXGB)

0.9128081340062908

In [26]:
XGB_y_pred = best_model_XGB.predict(test_X)
XGBboost_y_score = best_model_XGB.predict_proba(test_X)[:, 1]
fprxgb,tprxgb,threshold = roc_curve(test_y,XGBboost_y_score)
auc(fprxgb,tprxgb)

0.8877611940298507

In [27]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob =best_model_XGB.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
    best_GB = GradientBoostingClassifier(learning_rate=0.005,  
                                           n_estimators=278,  
                                           min_samples_split=38,  
                                           min_samples_leaf=27,  
                                           max_depth=3,  
                                           max_features=2,  
                                           subsample=0.7,
                                           random_state=102030)  

    best_GB.fit(train_X, train_y)  

GradientBoostingClassifier(learning_rate=0.005, max_features=2,
                           min_samples_leaf=27, min_samples_split=38,
                           n_estimators=278, random_state=102030,
                           subsample=0.7)

In [30]:
test_gb_pred_proba = best_GB.predict_proba(test_X)[:, 1]
test_auc = roc_auc_score(test_y, test_gb_pred_proba)
GBboost_y_pred=best_GB.predict(test_X)
print("testAUC:", test_auc)
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

testAUC: 0.8906467661691542


0.8906467661691542

In [31]:
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

0.8906467661691542

In [32]:
GBboost_train_y_pred=best_GB.predict(train_X)
train_GB_pred_proba = best_GB.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_GB_pred_proba)
print("Train AUC:", train_auc )

Train AUC: 0.9233230926779313


In [33]:
fprGB,tprGB,threshold = roc_curve(train_y,train_GB_pred_proba)
auc(fprGB,tprGB)

0.9233230926779313

In [34]:
best_GB_y_score =best_GB.predict_proba(test_X)[:, 1]

In [35]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV


best_params_LGBM = {
    'lambda_l1': 0.7,
    'lambda_l2': 1e-05,
    'learning_rate': 0.005,
    'n_estimators': 600,
    'max_depth': 4,
    'num_leaves': 12,
    'max_bin': 14,
    'min_data_in_leaf': 32,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.6,
    'bagging_freq': 10,
    'random_state': 123,
    'verbose': -1
}


best_LGBM = lgb.LGBMClassifier(**best_params_LGBM)
best_LGBM.fit(train_X, train_y)

LGBMClassifier(bagging_fraction=0.6, bagging_freq=10, feature_fraction=0.9,
               lambda_l1=0.7, lambda_l2=1e-05, learning_rate=0.005, max_bin=14,
               max_depth=4, min_data_in_leaf=32, n_estimators=600,
               num_leaves=12, random_state=123, verbose=-1)

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
train_y_pred = best_LGBM.predict(train_X)
train_accuracy = accuracy_score(train_y, train_y_pred)
print("Training Accuracy:", train_accuracy)
train_y_lgbm_prob = best_LGBM.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_y_prob)
print("Training AUC：", train_auc)

Training Accuracy: 0.8378378378378378
Training AUC： 0.9128081340062908


In [37]:
y_score_LGBM = best_LGBM.predict_proba(test_X)[:, 1]
LGBM_y_pred=best_LGBM.predict(test_X)
test_accuracy = accuracy_score(test_y, LGBM_y_pred)
print("Test_accuracy:", test_accuracy)
fprlgbm, tprlgbm, threshold = roc_curve(test_y, y_score_LGBM)
test_auc = roc_auc_score(test_y, y_score_LGBM )

print("Test AUC:", test_auc)

Test_accuracy: 0.823943661971831
Test AUC: 0.8920398009950249


In [38]:
fprLGBM, tprLGBM, thresholds= roc_curve(train_y, train_y_lgbm_prob)
fprlgbm, tprlgbm, thresholds = roc_curve(test_y, y_score_LGBM)

In [ ]:
# The following is the DeLong test for LR (R+M+P) and ML
# In R, the predicted probabilities from LR(R, M, and P) has been saved.

In [39]:
y_pred_LR = data_test.iloc[:, -1] 

In [40]:
y_pred_LR

0      0.853261
1      0.717861
2      0.079184
3      0.499924
4      0.502369
         ...   
137    0.267086
138    0.044056
139    0.836297
140    0.040276
141    0.725574
Name: predicted_probability, Length: 142, dtype: float64

In [41]:
test_nomo_auc = roc_auc_score(test_y, y_pred_LR)
test_nomo_auc

0.8865671641791044

In [52]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
from sklearn import metrics

class DelongTest():
    def __init__(self, preds1, preds2, label, threshold=0.05):
        '''
        preds1: the output of model1
        preds2: the output of model2
        label : the actual label
        '''
        self._preds1 = preds1
        self._preds2 = preds2
        self._label = label
        self.threshold = threshold
        self.result = self._show_result()

    def _auc(self, X, Y) -> float:
        return 1 / (len(X) * len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self, X, Y) -> float:
        '''
        Mann-Whitney statistic
        '''
        return .5 if Y == X else int(Y < X)

    def _structural_components(self, X, Y) -> list:
        V10 = [1 / len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1 / len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self, V_A, V_B, auc_A, auc_B) -> float:
        return 1 / (len(V_A) - 1) * sum([(a - auc_A) * (b - auc_B) for a, b in zip(V_A, V_B)])
    
    def _z_score(self, var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B) / ((var_A + var_B - 2 * covar_AB) ** (.5) + 1e-8)

    def _group_preds_by_label(self, preds, actual) -> list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        # Compute entries of covariance matrix S (covar_AB = covar_BA)
        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) / len(V_A10) + self._get_S_entry(V_A01, V_A01, auc_A, auc_A) / len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) / len(V_B10) + self._get_S_entry(V_B01, V_B01, auc_B, auc_B) / len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) / len(V_A10) + self._get_S_entry(V_A01, V_B01, auc_A, auc_B) / len(V_A01))

        # Two-tailed test
        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z)) * 2

        return z, p

    def _show_result(self):
        z, p = self._compute_z_p()
        result = f"z score = {z:.5f};\np value = {p:.5f};\n"
        if p < self.threshold:
            result += "There is a significant difference"
        else:
            result += "There is NO significant difference"
        print(result)
        return result
    
    def __str__(self):
        return self.result

In [53]:
DelongTest(y_pred_LR,RF_y_score,test_y)

z score = 0.47526;
p value = 0.63460;
There is NO significant difference


In [54]:
DelongTest(y_pred_LR,XGBboost_y_score,test_y)

z score = -0.09148;
p value = 0.92711;
There is NO significant difference


In [55]:
DelongTest(y_pred_LR,test_gb_pred_proba ,test_y)

z score = -0.32173;
p value = 0.74766;
There is NO significant difference


In [56]:
DelongTest(y_pred_LR,y_score_LGBM ,test_y)

z score = -0.36484;
p value = 0.71523;
There is NO significant difference
